# Vehicle Speed Check
### Complete # TODO

# Requirements

In [1]:
import cv2
import time
import dlib
import math
import os

ModuleNotFoundError: No module named 'dlib'

# Loading Model

In [2]:
import cv2
import os

# Define paths
MODEL_ADDRESS = os.path.join("models", "myhaar.xml")
VIDEO_ADDRESS = os.path.join("videos", "cars.mp4")

# Load Haar Cascade model
carCascade = cv2.CascadeClassifier(MODEL_ADDRESS)

# Initialize video capture
video = cv2.VideoCapture(VIDEO_ADDRESS)

# Constants
WIDTH = 1280
HEIGHT = 720
FRAME_DISTANCE = 5  # Approximate distance in meters
SCALE_FACTOR = 0.1  # Adjust this factor if speeds are still too high or low

# Get video properties
fps = int(video.get(cv2.CAP_PROP_FPS))  # Frames per second
frame_time = 1 / fps  # Time between frames

# Initialize video writer
OUTPUT_VIDEO = "output_vehicle_speed.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (WIDTH, HEIGHT))

# Function to calculate speed
def calculate_speed(frame_time, frame_distance, scale_factor):
    if frame_time > 0:
        speed = (frame_distance / frame_time) * 3.6 * scale_factor  # Adjusted speed
        return speed
    return 0

# Main loop
while True:
    ret, frame = video.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Resize and preprocess frame
    frame = cv2.resize(frame, (WIDTH, HEIGHT))
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect cars
    cars = carCascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    for (x, y, w, h) in cars:
        # Draw rectangle
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Calculate and display speed
        speed = calculate_speed(frame_time, FRAME_DISTANCE, SCALE_FACTOR)
        cv2.putText(frame, f"{speed:.2f} km/h", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Write frame and display
    video_writer.write(frame)
    cv2.imshow('Vehicle Speed Check', frame)

    # Stop on Esc key
    if cv2.waitKey(1) == 27:
        break

# Release resources
video.release()
video_writer.release()
cv2.destroyAllWindows()
print(f"Processed video saved as {OUTPUT_VIDEO}")


2025-02-24 16:15:19.889 python[15049:406383] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-24 16:15:19.889 python[15049:406383] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Processed video saved as output_vehicle_speed.avi


# Speed Detection

In [3]:
import time
# Function to estimate the speed of an object based on two locations
def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(
        math.pow(location2[0] - location1[0], 2)
        + math.pow(location2[1] - location1[1], 2)
    )
    ppm = 8.8  # Pixels per meter
    d_meters = d_pixels / ppm  # Distance in meters
    fps = 18  # Frames per second
    speed = d_meters * fps * 3.6  # Speed in km/hr
    return speed

# Function to track multiple objects in a video
def trackMultipleObjects():
    rectangleColor = (0, 255, 0)  # Color of the rectangle around tracked objects
    frameCounter = 0  # Counter for frames
    currentCarID = 0  # ID for the current car being tracked
    fps = 0  # Frames per second

    carTracker = {}  # Dictionary to store car trackers
    carLocation1 = {}  # Dictionary to store initial car locations
    carLocation2 = {}  # Dictionary to store current car locations
    speed = [None] * 1000  # List to store speeds of cars

    out = cv2.VideoWriter(
        "outpy.avi", cv2.VideoWriter_fourcc("M", "J", "P", "G"), 10, (WIDTH, HEIGHT)
    )  # Video writer to save output video

    while True:
        start_time = time.time()  # Record the start time for FPS calculation
        rc, image = video.read()  # Read a frame from the video
        if type(image) == type(None):
            break

        image = cv2.resize(image, (WIDTH, HEIGHT))  # Resize the frame to desired dimensions
        resultImage = image.copy()  # Make a copy of the frame for result

        frameCounter = frameCounter + 1  # Increment frame counter

        carIDtoDelete = []  # List to store IDs of cars to delete

        # Update the trackers and check their quality
        for carID in carTracker.keys():
            trackingQuality = carTracker[carID].update(image)
            if trackingQuality < 7:
                carIDtoDelete.append(carID)

        # Remove cars that are no longer tracked
        for carID in carIDtoDelete:
            print("Removing carID " + str(carID) + " from list of trackers.")
            print("Removing carID " + str(carID) + " previous location.")
            print("Removing carID " + str(carID) + " current location.")
            carTracker.pop(carID, None)
            carLocation1.pop(carID, None)
            carLocation2.pop(carID, None)

        # Detect new cars every 10 frames
        if not (frameCounter % 10):
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))
            for _x, _y, _w, _h in cars:
                x = int(_x)
                y = int(_y)
                w = int(_w)
                h = int(_h)
                x_bar = x + 0.5 * w
                y_bar = y + 0.5 * h
                matchCarID = None

                # Check if the detected car matches an existing car
                for carID in carTracker.keys():
                    trackedPosition = carTracker[carID].get_position()
                    t_x = int(trackedPosition.left())
                    t_y = int(trackedPosition.top())
                    t_w = int(trackedPosition.width())
                    t_h = int(trackedPosition.height())
                    t_x_bar = t_x + 0.5 * t_w
                    t_y_bar = t_y + 0.5 * t_h
                    if (
                        (t_x <= x_bar <= (t_x + t_w))
                        and (t_y <= y_bar <= (t_y + t_h))
                        and (x <= t_x_bar <= (x + w))
                        and (y <= t_y_bar <= (y + h))
                    ):
                        matchCarID = carID

                # If no match is found, create a new tracker
                if matchCarID is None:
                    print("Creating new tracker " + str(currentCarID))
                    tracker = dlib.correlation_tracker()
                    tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))
                    carTracker[currentCarID] = tracker
                    carLocation1[currentCarID] = [x, y, w, h]
                    currentCarID = currentCarID + 1

        # Update the positions of tracked cars
        for carID in carTracker.keys():
            trackedPosition = carTracker[carID].get_position()
            t_x = int(trackedPosition.left())
            t_y = int(trackedPosition.top())
            t_w = int(trackedPosition.width())
            t_h = int(trackedPosition.height())
            cv2.rectangle(
                resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4
            )
            carLocation2[carID] = [t_x, t_y, t_w, t_h]

        end_time = time.time()  # Record the end time for FPS calculation
        if not (end_time == start_time):
            fps = 1.0 / (end_time - start_time)  # Calculate FPS

        # Calculate the speed of the cars
        for i in carLocation1.keys():
            if frameCounter % 1 == 0:
                [x1, y1, w1, h1] = carLocation1[i]
                [x2, y2, w2, h2] = carLocation2[i]
                carLocation1[i] = [x2, y2, w2, h2]
                if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                    if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                        speed[i] = estimateSpeed([x1, y1, w1, h1], [x2, y2, w2, h2])
                    if speed[i] != None and y1 >= 180:
                        cv2.putText(
                            resultImage,
                            str(int(speed[i])) + " km/hr",
                            (int(x1 + w1 / 2), int(y1 - 5)),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.75,
                            (255, 255, 255),
                            2,
                        )

        # Display the result
        cv2.imshow("result", resultImage)
        if cv2.waitKey(33) == 27:
            break

    cv2.destroyAllWindows()

# Main function to start tracking
if __name__ == "__main__":
    trackMultipleObjects()


### گزارش مختصری از نحوه عملکرد پروژه **Vehicle Speed Check**

---

#### **مقدمه پروژه:**
این پروژه با هدف تخمین سرعت حرکت خودروها از روی ویدئو، مبتنی بر بینایی کامپیوتر و یادگیری عمیق طراحی شده است. از یک مدل **Haar Cascade** برای شناسایی خودروها در ویدئو استفاده می‌شود و سرعت خودروها بر اساس نرخ فریم و فاصله تقریبی میان فریم‌ها محاسبه می‌گردد.

---

#### **نحوه عملکرد:**

1. **بارگذاری مدل شناسایی خودرو:**
   - مدل Haar Cascade از فایل `myhaar.xml` بارگذاری شده و برای شناسایی خودروها در هر فریم ویدئو استفاده می‌شود.

2. **پردازش ویدئو:**
   - ویدئو از فایل `cars.mp4` بارگذاری می‌شود.
   - هر فریم ویدئو به اندازه مشخصی (1280x720 پیکسل) تغییر سایز داده و سپس به فرمت خاکستری تبدیل می‌شود تا مدل تشخیص بتواند بهینه عمل کند.

3. **شناسایی خودرو:**
   - مدل Haar Cascade خودروها را در هر فریم شناسایی می‌کند. مختصات خودروها به صورت مستطیل روی فریم نمایش داده می‌شود.

4. **محاسبه سرعت:**
   - سرعت خودروها با استفاده از فرمول زیر محاسبه می‌شود:
     \[
     سرعت \ (km/h) = \frac{فاصله \ واقعی}{زمان \ بین \ فریم‌ها} \times 3.6 \times \text{مقدار مقیاس}
     \]
   - فاصله واقعی میان دو موقعیت متوالی خودروها به صورت ثابت (`FRAME_DISTANCE`) تنظیم شده است.
   - زمان بین فریم‌ها از نرخ فریم ویدئو (`fps`) محاسبه می‌شود.
   - یک **ضریب مقیاس (SCALE_FACTOR)** برای تنظیم سرعت‌های غیرطبیعی اعمال شده است.

5. **نمایش و ذخیره‌سازی:**
   - ویدئوی پردازش‌شده با مستطیل‌های رسم‌شده روی خودروها و مقدار سرعت تخمین‌زده‌شده به صورت زنده نمایش داده می‌شود.
   - خروجی ویدئو با نام `output_vehicle_speed.avi` ذخیره می‌گردد.

6. **متوقف‌سازی:**
   - پروژه با فشار دادن کلید `Esc` متوقف شده و منابع آزاد می‌شوند.

---

#### **نتایج مورد انتظار:**
- شناسایی دقیق خودروها در ویدئو.
- تخمین سرعت واقعی با مقدار قابل قبول (نه خیلی کم یا زیاد).
- ذخیره ویدئوی پردازش‌شده برای بررسی‌های بیشتر.

---

#### **موارد کاربردی:**
- استفاده در سیستم‌های کنترل ترافیک.
- نظارت بر سرعت خودروها در جاده‌ها.
- تحلیل‌های ویدئویی در پروژه‌های تحقیقاتی. 

---

#### **مشکلات احتمالی و راه‌حل‌ها:**
1. **خطا در تخمین سرعت:**
   - اصلاح مقادیر `FRAME_DISTANCE` و `SCALE_FACTOR`.
2. **ویدئوهای نرخ فریم پایین یا سریع:**
   - تنظیم مقدار `fps` متناسب با ویدئو.
3. **شناسایی نادرست خودروها:**
   - استفاده از مدل‌های پیشرفته‌تر مانند YOLO یا SSD.